In [ ]:
# Install dependencies
!uv pip install matplotlib

In [ ]:
# Parse a fit file
from power_metrics_lib.file_parsers import parse_activity_file
from power_metrics_lib.models import Activity

activity: Activity = parse_activity_file("../tests/files/file.fit")

In [ ]:
# Plot the power data
import matplotlib.pyplot as plt

plt.plot(activity.power)
plt.ylabel("watt")
plt.xlabel("duration")
plt.show()

In [ ]:
# Calculate some initial metrics:
import power_metrics_lib.calculate_metrics as pm

np = pm.calculate_normalized_power(activity.power)
ap = pm.calculate_average_power(activity.power)
print(f"normalized power (np): {np}")
print(f"average power (ap): {ap}")

In [ ]:
# Based on np, we can calculate the intensity_factor given your ftp:
ftp = 236
intensity_factor = pm.calculate_intensity_factor(np, ftp)
print(f"intensity_factor: {intensity_factor}")

In [ ]:
# More metrics:
duration = pm.calculate_duration(activity.timestamps)
tss = pm.calculate_training_stress_score(
    normalized_power=np,
    ftp=ftp,
    duration=duration["seconds"],
    intensity_factor=intensity_factor,
)
print(
    f"duration:\n \tin seconds: {duration['seconds']}\n\tin hhmmss: {duration['hh:mm:ss']}"
)
print(f"total stress score: {tss}")

In [ ]:
# Total work and max power:
total_work = pm.calculate_total_work(activity.power)
max_power = pm.calculate_max_power(activity.power)
print(f"total work: {total_work}")

print(f"max power: {max_power}")

In [ ]:
# Parse workout file:
from power_metrics_lib.file_parsers import parse_workout_file
from power_metrics_lib.models import Workout

workout: Workout = parse_workout_file("../tests/files/mosaic.zwo")
print(workout)

In [ ]:
# Create activity from workout:
from power_metrics_lib.transformers import transform_workout_to_activity

workout_activity: Activity = transform_workout_to_activity(workout, ftp)

plt.plot(workout_activity.power)
plt.ylabel("watt")
plt.xlabel("duration")
plt.show()

In [ ]:
# Calculate metrics on workout:
workout_np = pm.calculate_normalized_power(workout_activity.power)
workout_ap = pm.calculate_average_power(workout_activity.power)
print(f"workout_np: {workout_np}")
print(f"workout_ap: {workout_ap}")

In [ ]:
workout_duration = pm.calculate_duration(workout_activity.timestamps)
workout_intensity_factor = pm.calculate_intensity_factor(np, ftp)
workout_tss = pm.calculate_training_stress_score(normalized_power=workout_np, ftp=ftp, intensity_factor=workout_intensity_factor, duration=workout_duration["seconds"])
print(f"workout_duration: {workout_duration}")
print(f"workout_intensity_factor: {workout_intensity_factor}")
print(f"workout_tss: {workout_tss}")

In [ ]:
# Compare
print("\t\tActivity\tWorkout")
print(f"duration:\t{duration["hh:mm:ss"]}\t\t{workout_duration["hh:mm:ss"]}")
print(f"ap:\t\t{round(ap,2)}\t\t{round(workout_ap, 2)}")
print(f"np:\t\t{round(np,2)}\t\t{round(workout_np, 2)}")
print(f"tss:\t\t{round(tss,2)}\t\t{round(workout_tss, 2)}")
print(f"if:\t\t{round(intensity_factor,2)}\t\t{round(workout_intensity_factor, 2)}")

In [ ]:
# Plot both
plt.figure(figsize=(15,5), layout="constrained")
plt.plot(activity.power, label="activity")
plt.plot(workout_activity.power, label="workout")
plt.ylabel("watt")
plt.xlabel("duration")
plt.legend()